In [1]:
# import dependencies
import bs4
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
# from selenium import webdriver
# from lxml import html
import csv
from datetime import datetime, date, timedelta
from pytz import timezone
import pandas as pd

In [ ]:
# url of ucsc college 9 & 10 dining hall
url = 'https://nutrition.sa.ucsc.edu/nutframe.asp?sName=UC+Santa+Cruz+Dining&locationNum=40&locationName=Colleges+Nine+%26+Ten+Dining+Hall&naFlag=1'

In [ ]:
# scrap the menu data from the webpage
# since the menu data is cannot be scrapped from the html file directly
# we have to make additional requests to get the frame page contents 
with requests.Session() as session:
    response = session.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    for frame in soup.select("frameset frame"):
        frame_url = urljoin(url, frame["src"])
        response = session.get(frame_url)
        frame_soup = BeautifulSoup(response.content, 'html.parser') 
        # print(frame_soup.prettify())

In [ ]:
# extract both the meal name and food name from html
meal_name = frame_soup.find_all('div', attrs={'class': 'menusampmeals'})
food_name = frame_soup.find_all('div', attrs={'class': 'menusamprecipes'})

In [ ]:
# convert the name of the meal type in html form to a list form
meal_type = [item.string for item in meal_name]
print(meal_type)

In [ ]:
# convert the food name in html form to a list form
food_list = [item.string for item in food_name]
print(food_list)
print(len(food_list))

In [ ]:
# In case of dupliate food name, the list should be converted to a set
food_list = sorted(list(set(food_list)))
print(food_list)
print(len(food_list))

In [ ]:
# preference list, labeled by myself
# pref_1 = [1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0 ]
# pref_2 = [1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0]
pref_3 = [0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1]

In [ ]:
# timezone setting
us_pacific = timezone('US/Pacific')
time = datetime.now(us_pacific)
us_time = time.strftime('%Y-%m-%d')
us_time

In [ ]:
# write the food list to a csv file
csv_path = '/Users/7w0r4ng3s/Desktop/menu_scraping/{}.csv'.format(us_time)
csv_path

In [ ]:
def write_full():
    with open(csv_path, "w") as output:
        writer = csv.writer(output, lineterminator='\n')
        for key, val in zip(food_list, pref_3):
            writer.writerow([key, val])
write_full()

In [13]:
df = pd.read_csv('2018-09-07.csv', names=['food', 'pref'])
df.to_csv('2018-09-07.csv')

In [14]:
df_new = pd.read_csv('2018-09-07.csv')

In [15]:
df_new

,Unnamed: 0,food,pref
0,0,Blueberry Pancakes,1
1,1,Cage Free Scrambled Eggs,1
2,2,Hard-boiled Cage Free Eggs,1
3,3,Natural BridgesTofu Scramble,0
4,4,Oatmeal Gluten-Free,0
5,5,Sausage Patties,1
6,6,Steamed Rice,0
7,7,Tator Tots,0
8,8,N.E.Clam Chowder,0
9,9,Baked Salmon with Avocado Chimichurri,1


In [9]:
df.sort_values(by=['food'], ascending=[True])

,food,pref
14,Armenian Salad,0
9,Baked Salmon with Avocado Chimichurri,1
15,Beef and Lamb Gyro,0
10,"Black Rice, Beet and Kale Salad",1
0,Blueberry Pancakes,1
1,Cage Free Scrambled Eggs,1
24,Cajun Bayou Bar,0
25,Cajun Coleslaw,1
26,Cajun Rice,0
20,Chef's Special,1


In [ ]:
df = df.reset_index().drop_duplicates()
df